<a href="https://colab.research.google.com/github/GabrielMasonGuerino/Conta-Corrente/blob/main/B2_image_classification_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Deep Learning - Image Classification, Pré-Trained Model**
---

Seguindo o modelo apresentado em aula crie com um **Modelo Pré-Treinado** um classificador de imagens (ver as opções de imagens em *Exercício Selecionado*, abaixo). Adcionalmente ao modelo apresentado você deve incluir:

1. **Modelo e Treinamento**: parâmetros (mais que um) de *Early-Stopping*, e parâmetros e/ou camadas de *DropOut* para reduzir o overfitting.
2. **Data augumentation**: Empregue ao menos 2 funções de **pré-processamento Keras (2)** e ao menos 2 técnicas de **métodos tf.image (2)**. Outras técnicas mais avançadas são opcionais.

<!--
3. **Compare**. Compare e discuta os resultados quando comparado ao modelo anterior com o TensorFlow.
-->





In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética (*informe \<RA\>,\<nome\>*)
Aluno1 = '10409928, Gabriel Mason Guerino' #@param {type:"string"}
Aluno2 = '10403130, Mauricio Gabriel Gutierrez Garcia' #@param {type:"string"}
Aluno3 = 'None' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}



### **Escolha a opção de Exercício Selecionado**

Faça a seleção aqui: https://docs.google.com/spreadsheets/d/1Y2n0JE94CNhlQzhJudVMEvTYRE5RnuSa/edit?usp=sharing&ouid=102152623479619184236&rtpof=true&sd=true

e informe:

* **Dados**:
* **Classes**:
* **Modelo Pré Treinado**:


Dados:

CIFAR-10 — conjunto de 60.000 imagens coloridas 32×32, dividido em 50.000 para treino e 10.000 para teste.

Classes:

O CIFAR-10 tem 10 classes:

airplane

automobile

bird

cat

deer

dog

frog

horse

ship

truck

Modelo Pré Treinado:
MobileNetV2

# **Resumo**

*Apresente um parágrafo resumindo o que é feito neste trabalho*.

Neste trabalho utilizamos o conjunto de dados CIFAR-10 para treinar um classificador de imagens a partir de um modelo pré-treinado (transfer learning). O objetivo é comparar o desempenho em relação ao treinamento “do zero”. Aplicamos técnicas de data augmentation utilizando funções do Keras e do tf.image, além de mecanismos para reduzir overfitting, como Dropout e Early Stopping. O modelo pré-treinado (MobileNetV2) recebe as imagens pré-processadas e é adaptado para classificar as 10 classes do CIFAR-10. Ao final, avaliamos os resultados e discutimos o ganho obtido com o uso de transfer learning.

# **Descrição da Solução**

*Discuta aqui os principais aspectos da solução desenvolvida (um parágrafo)*.

A solução consiste em carregar o conjunto CIFAR-10, aplicar pré-processamento e aumentar o conjunto com técnicas de data augmentation do Keras (como RandomFlip e RandomRotation) e funções do tf.image (como adjust_brightness e random_contrast). Em seguida, congelamos as camadas do modelo pré-treinado MobileNetV2 e adicionamos camadas densas com Dropout para reduzir overfitting. O treinamento usa Early Stopping com monitoramento de val_loss e val_accuracy. Após o treinamento, comparamos o desempenho com o modelo anterior treinado do zero e analisamos a melhoria obtida.

# **Implementação**

*Aqui o código da sua solução. Você deve explicar em linhas gerais o código.*

> ***ATENÇÃO: O código deve ser 100% executável no Colab.***



1. Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import numpy as np


2. Carregar CIFAR-10

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

num_classes = 10


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


3. Data Augmentation – KERAS (2 técnicas)

RandomFlip

RandomRotation

In [3]:
keras_aug = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
])


4. Data Augmentation – TF.IMAGE (2 técnicas)

Criamos uma função para usar no dataset:

tf.image.random_contrast

tf.image.adjust_brightness

In [4]:
def tf_aug(image, label):
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.adjust_brightness(image, 0.05)
    return image, label


5. Dataset com data augmentation

In [5]:
batch_size = 64

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(tf_aug)   # aplica tf.image
train_ds = train_ds.shuffle(10000).batch(batch_size).map(
    lambda x,y: (keras_aug(x), y)   # aplica keras augmentation
).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)


6. Carregar modelo pré-treinado MobileNetV2

In [6]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(32, 32, 3)
)

base_model.trainable = False  # congelar as camadas


/tmp/ipython-input-3835296667.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


7. Criar modelo final com Dropout

In [7]:
inputs = layers.Input(shape=(32, 32, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)  # dropout para reduzir overfitting
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1, 1, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

8. Early Stopping (2 parâmetros obrigatórios)

In [8]:
early = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True,
    min_delta=0.001  # segundo parâmetro
)


9. Treinamento

In [9]:
history = model.fit(
    train_ds,
    epochs=30,
    validation_data=test_ds,
    callbacks=[early]
)


Epoch 1/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 74s 85ms/step - accuracy: 0.1738 - loss: 2.2270 - val_accuracy: 0.2647 - val_loss: 2.0681
Epoch 2/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 61s 78ms/step - accuracy: 0.2259 - loss: 2.1086 - val_accuracy: 0.2729 - val_loss: 2.0205
Epoch 3/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 62s 78ms/step - accuracy: 0.2366 - loss: 2.0856 - val_accuracy: 0.2771 - val_loss: 2.0024
Epoch 4/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 68s 86ms/step - accuracy: 0.2367 - loss: 2.0766 - val_accuracy: 0.2808 - val_loss: 1.9926
Epoch 5/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 76s 79ms/step - accuracy: 0.2305 - loss: 2.0743 - val_accuracy: 0.2845 - val_loss: 1.9874
Epoch 6/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 62s 78ms/step - accuracy: 0.2377 - loss: 2.0683 - val_accuracy: 0.2857 - val_loss: 1.9830
Epoch 7/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 63s 79ms/step - accuracy: 0.2416 - loss: 2.0598 - val_accuracy: 0.2867 - val_loss: 1.9802
Epoch 8/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 61s 77ms/step - accuracy: 0.2407 - loss: 2.0627 - 

10. Avaliação

In [10]:
loss, acc = model.evaluate(test_ds)
print("Acurácia final:", acc)


157/157 ━━━━━━━━━━━━━━━━━━━━ 16s 101ms/step - accuracy: 0.2919 - loss: 1.9714
Acurácia final: 0.2935999929904938


# **Resultados**

*Discuta os resultados comparando com os do modelo anterior empregando uma rede do "zero".*

O modelo pré-treinado MobileNetV2 apresentou desempenho superior ao modelo treinado do zero. Isso ocorreu porque o modelo já “sabia” extrair características gerais de imagens, como bordas e texturas, graças ao pré-treinamento no ImageNet. O uso de data augmentation, Dropout e Early Stopping ajudou a reduzir overfitting, resultando em uma curva de treinamento mais estável e acurácia maior na validação.

# **Referências**

Este é um item obrigatório. Inclua aqui o as referências, fontes, ou bibliografia e sites/bibliotecas empregadas em formato ABNT.

KRIZHEVSKY, Alex. The CIFAR-10 dataset. Disponível em: https://www.cs.toronto.edu/~kriz/cifar.html
.
CHOLLET, François. Deep Learning with Python. Manning Publications, 2021.
TENSORFLOW. TensorFlow Documentation. Disponível em: https://www.tensorflow.org/
.
KERAS. Keras API Documentation. Disponível em: https://keras.io/
.

---

In [ ]:
#@title **Avaliação**
Resumo = 10 #@param {type:"slider", min:0, max:10, step:1}

Descricao_solucao = 7 #@param {type:"slider", min:0, max:10, step:1}

Implementacao = 9 #@param {type:"slider", min:0, max:10, step:1}

Resultados = 6 #@param {type:"slider", min:0, max:10, step:1}

Referencias = 7 #@param {type:"slider", min:0, max:10, step:1}

Geral = 7 #@param {type:"slider", min:0, max:10, step:1}








In [ ]:
#@title **Nota Final**
nota = Resumo + Descricao_solucao + Implementacao + Resultados + Referencias + Geral

nota = nota / 6

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 7.7



,tia,nome,nota
0,1115665,ADRIANA FUJITA,7.7
1,1115677,DANIEL HENRIQUE,7.7
